### 기본 설정

- 필수 모듈 불러오기
- 그래프 출력 관련 기본 설정 지정

In [ ]:
# 파이썬 ≥3.5 필수 (파이썬 3.7 추천)
import sys
assert sys.version_info >= (3, 5) 

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "decision_trees"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# **문제 1: 6장 연습문제 8번**

랜덤 포레스트를 직접 구현한 다음에 7장에서 소개한 사이킷런의 랜덤 포레스트 모델을 사용하는 것과의 성능을 비교하라.

## **직접구현 랜덤 포레스트(accuracy_score: 0.872)**

**make_moons 데이터셋을 생성**

 - `make_moons(n_samples=10000, noise=0.4)` 이용
 - 결과를 일정하게 만들기 위해 `random_state=42` 추가

In [ ]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

**make_moons 데이터셋을 train 세트와 test 세트 분할**

 - `train_test_split()` 사용

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**`DecisionTreeClassifier`의 최적 하이퍼 파라미터 찾기: 교차 검증과 그리드 탐색 활용**
 - `GridSearchCV` 활용
 - `max_leaf_nodes` 다양한 값 사용

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=1, cv=3)

grid_search_cv.fit(X_train, y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 882 out of 882 | elapsed:    9.0s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,


`max_leaf_nodes=17`, `min_samples_split=2`일 때, 최적의 하이퍼 파라미터 값이 나옴.

In [ ]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=17,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

**랜덤 포레스트 만들기**
 - 각각 무작위로 선택된 100개의 `n_instances`와 1,000개의 `n_trees` 생성
 - 사이킷런의 `ShuffleSplit` 클래스 활용

In [ ]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)

for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

**앞에서 찾은 최적의 하이퍼 파라미터 값을 사용**
 - `clone(grid_search_cv.best_estimator_)` 이용

각각의 서브 데이터셋에 결정 트리 훈련, 테스트 세트에서 1,000개의 결정트리 평가

더 작은 데이터셋에서 훈련되었기 때문에 성능이 이전보다 낮다.

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.base import clone
# clone: 동일 파라미터를 가지고 새로운 추정치 생성

forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.8054499999999999

**각 테스트 세트 샘플에 1,000개의 결정 트리 예측 생성**
 - SciPy의 `mode()`를 사용하여 다수결 예측(_majority-vote predictions_)이 만들어진다.

In [ ]:
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

In [ ]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

테스트 세트에서 예측을 평가하면 첫 번째 모형보다 조금 더 높은 정확도를 얻는다.

In [ ]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.872

In [ ]:
np.sum(y_pred == y_pred_majority_votes) / len(y_pred)

0.8995

## **사이킷런 랜덤 포레스트(accuracy_score: 0.8715)**

**사이킷런을 이용하여 랜덤 포레스트 모델 생성**
 - `n_estimators=1000, max_leaf_nodes=17, random_state=42` 설정

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=1000, max_leaf_nodes=17, random_state=42)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

사이킷런을 활용한 랜덤 포레스트 예측 값이 위의 직접 랜덤 포레스트 모델을 구현한 예측 값과 비슷하다.

In [ ]:
accuracy_score(y_test, y_pred_rf)

0.8715

## **두 모델의 예측 값 비교(98.25% 일치)**

직접 구현한 `y_pred_majority_votes`의 예측 값과 사이킷런의 `y_pred_rf` 예측 값을 비교한다.

0.9825만큼 일치하는 것을 확인할 수 있다.

In [ ]:
np.sum(y_pred_majority_votes == y_pred_rf) / len(y_pred)

0.9825

# **문제 2: 7장 연습문제 9번**

스태킹 모델을 직접 구현한 후 7장 강의노트에서 소개한 사이킷런의 스태킹 모델을 사용하는 것과의 성능을 비교하라.

#### **MNIST 데이터셋 가져오기**

아래 코드는 MNIST 사진에 포함된 픽셀별 중요도를 이미지로 보여준다.

In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
mnist.target = mnist.target.astype(np.uint8)

MNIST 데이터셋을 훈련 세트, 검증 세트, 테스트 세트로 각각 10,000개, 10,000개, 10,000개로 나눈다.

ps. 훈련 세트도 10,000개인 이유는 모델 훈련이 너무 오래 걸려서 입니다.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split(
    mnist.data, mnist.target, test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=10000, random_state=42)

# 남는 훈련 세트가 50,000개의 이미지 파일
# 스태킹 모델 훈련이 오래 걸리기 때문에 10,000개로 설정
X_train, X_train, y_train, y_train = train_test_split(
    X_train, y_train, test_size=10000, random_state=42)

## **직접구현 스태킹 모델(accuracy_score: 0.9384)**

**Random Forest, Extra-Trees, SVM, MLP 모델 가져오기**

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [ ]:
# 각각의 모델 적용
random_forest_clf = RandomForestClassifier(n_estimators=20, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=20, random_state=42)
svm_clf = LinearSVC(max_iter=20, tol=20, random_state=42)
mlp_clf = MLPClassifier(random_state=42)

In [ ]:
estimators = [random_forest_clf, extra_trees_clf, svm_clf, mlp_clf]
for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(X_train, y_train)

Training the RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
Training the ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
      

**개별 실행을 통해 검증 세트 예측과 결과 예측으로 새 훈련 세트 작성**
 - 각 훈련 샘플은 이미지에 대한 `classifiers`의 예측 세트를 포함하는 벡터이며 대상은 이미지 클래스이다.
 - 새로운 훈련 세트에 대한 `classfier` 훈련

In [ ]:
X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

In [ ]:
X_val_predictions

array([[5., 5., 5., 5.],
       [8., 8., 8., 8.],
       [2., 2., 2., 2.],
       ...,
       [4., 7., 7., 7.],
       [6., 6., 6., 6.],
       [7., 7., 7., 7.]], dtype=float32)

In [ ]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_val_predictions, y_val)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=True, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
rnd_forest_blender.oob_score_

0.942

미세 조정을 통해 최적의 블렌더를 선택할 수 있다.

**`Blender` 훈련 후 모든 `Classifier`에 대해서 스태킹 모델 형성**

테스트 세트의 앙상블을 평가해보자.

테스트 세트의 각 이미지에 대해 모든 classifier을 사용하여 예측하고, 다음 예측을 blender에 공급하여 앙상블의 예측 값을 가져온다.

In [ ]:
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

In [ ]:
y_pred1 = rnd_forest_blender.predict(X_test_predictions)

In [ ]:
from sklearn.metrics import accuracy_score

예측한 값인 `accuracy_score`의 값이 0.9384가 나온다.

In [ ]:
accuracy_score(y_test, y_pred1)

0.9384

## **사이킷런 스태킹 모델(accuracy_score: 0.9491)**

**사용할 모든 `Classifier`를 import 한다.**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier

`RandomForest`, `LinearSVC` **모델 적용 `StackingClassifier`**

In [ ]:
estimators = [('rf', RandomForestClassifier(n_estimators=20, random_state=42)),
              ('svr', make_pipeline(StandardScaler(),
                                    LinearSVC(random_state=42)))]

clf = StackingClassifier(estimators=estimators, 
                         final_estimator=LogisticRegression())

In [ ]:
clf.fit(X_train, y_train).score(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

0.9292

예측 평가 값이 0.9292으로 직접 구현한 것 보다 점수가 낮다.

하지만 RandomForest, LinearSVC만 적용한 스태킹 모델이기 때문에 이전에 직접 만든 스태킹 모델에서 사용하는 모델들을 가져와서 다시 실행해보자.

`RandomForest`, `ExtraTrees`, `LinearSVC`, `MLP` **모델 적용 `StackingClassifier`**

In [ ]:
estimators = [('rf', RandomForestClassifier(n_estimators=20, random_state=42)),
              ('ex', ExtraTreesClassifier(n_estimators=20, random_state=42)),
              ('svm', LinearSVC(max_iter=20, tol=20, random_state=42)),
              ('mlp', MLPClassifier(random_state=42))]

clf = StackingClassifier(estimators=estimators, 
                         final_estimator=LogisticRegression())

In [ ]:
clf.fit(X_train, y_train).score(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9491

예측 평가 값이 0.9491으로 직접 구현한 것 보다 점수가 높다.

훈련 세트를 많이 줄였기 때문에 정확한 훈련이 되지 않았을 수도 있다.

마지막으로 두 모델을 비교하기 위해 `y_pred2` 값을 따로 저장한다.

In [ ]:
y_pred2 = clf.predict(X_test)

## **두 모델의 예측 값 비교(95.97% 일치)**

**직접 구현한** `y_pred1` **예측 값과 사이킷런의** `y_pred2` **예측 값을 비교한다.**

0.9597만큼 일치하는 것을 확인할 수 있다.

In [ ]:
np.sum(y_pred1 == y_pred2) / len(y_val)

0.9597